# Week 1 Exercise: Explain My Stacktrace

This notebook builds a small tool that takes a **Python stacktrace** and returns a clear, concise explanation plus likely fixes.
It uses the OpenAI API and simple prompt engineering.

**Tip:** Replace the sample stacktrace with your own error output.


In [2]:
# Imports
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI



In [4]:
# Load environment variables (.env)
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print('No API key found. Please add OPENAI_API_KEY to your .env file.')
elif api_key.strip() != api_key:
    print('API key has leading/trailing whitespace. Please remove it.')
else:
    print('API key looks good!')

openai = OpenAI(
    api_key=api_key,
    base_url='https://openrouter.ai/api/v1',
)


API key looks good!


In [5]:
# Model selection
MODEL = 'gpt-4.1-mini'


In [6]:
# Example stacktrace (replace with your own)
stacktrace = '''
Traceback (most recent call last):
  File "app.py", line 18, in <module>
    result = divide(10, 0)
  File "app.py", line 7, in divide
    return a / b
ZeroDivisionError: division by zero
'''


In [7]:
# Another example: TypeError (replace with your own)
stacktrace_type_error = '''
Traceback (most recent call last):
  File "app.py", line 21, in <module>
    print(len(42))
TypeError: object of type 'int' has no len()
'''


In [8]:
SYSTEM_PROMPT = '''
You are a helpful debugging assistant.
Explain Python stacktraces clearly and concisely.
Return in markdown with the following sections (as bullet points):
- Diagnosis (plain English)
- Root Cause Line (quote the line from the traceback if you can)
- Minimal Fix (1-2 lines of code)
- Confidence (Low/Medium/High)
- Next Step (one concrete action)
Keep it short and actionable.
'''

USER_PROMPT_PREFIX = '''
Here is a Python stacktrace. Explain what happened and how to fix it:

'''


In [9]:
# Simple root-cause extractor (best-effort)
def extract_root_cause_line(trace_text: str) -> str:
    lines = [ln.strip() for ln in trace_text.strip().splitlines() if ln.strip()]
    # Prefer the final exception line
    if lines:
        return lines[-1]
    return ''


In [10]:
def messages_for(trace_text: str):
    root_cause = extract_root_cause_line(trace_text)
    hint = ''
    if root_cause:
        hint = f"\n\nRoot cause line (detected): {root_cause}"
    return [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT_PREFIX + trace_text + hint},
    ]


In [11]:
# Non-streaming version
def explain_stacktrace(trace_text: str) -> str:
    response = openai.chat.completions.create(
        model=MODEL,
        messages=messages_for(trace_text),
        max_tokens=800,
    )
    return response.choices[0].message.content


In [13]:
# Streaming version (updates output as tokens arrive)
def explain_stacktrace_stream(trace_text: str):
    root_cause = extract_root_cause_line(trace_text)
    if root_cause:
        display(Markdown(f'**Detected Root Cause:** `{root_cause}`'))
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=messages_for(trace_text),
        stream=True,
        max_tokens=800,
    )
    response = ''
    display_handle = display(Markdown(''), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

explain_stacktrace_stream(stacktrace)


**Detected Root Cause:** `ZeroDivisionError: division by zero`

```markdown
- Diagnosis
The program tried to divide a number by zero, which is not allowed in Python and caused a runtime error.

- Root Cause Line
`ZeroDivisionError: division by zero`

- Minimal Fix
Add a check before division to avoid dividing by zero:
```python
def divide(a, b):
    if b == 0:
        return "Error: Cannot divide by zero"
    return a / b
```

- Confidence
High

- Next Step
Modify the divide function to handle zero divisor cases gracefully and test with zero and non-zero values.
```

In [ ]:
# Or use the non-streaming version:
# display(Markdown(explain_stacktrace(stacktrace)))
